![Machine Learning Lab](banner.jpg)


# Laboratorio 3 Actividad


## Instrucciones generales


1. Esta actividad debe ser entregada por BN y es de carácter individual. No se permite entregar el laboratorio después de la fecha establecida.
2. Al responder las preguntas de las actividades por favor marcar las respuestas con la sección a la que corresponden, por ejemplo: `## Exploración de datos 2.1`. Es preferible que esto lo hagan con secciones de MarkDown.
3. Por favor nombrar el archivo de acuerdo al siguiente formato `{email}_lab3.ipynb`.
4. Si tienen alguna duda pueden escribirme a mi correo `j.rayom@uniandes.edu.co` o contactarme directamente por Teams

Para este laboratorio deben usar el siguiente dataset: `https://www.kaggle.com/datasets/vishardmehta/heart-risk-progression-dataset`

### Objetivos

1. Comprender la estructura y características del dataset mediante la exploración y visualización de datos para identificar patrones relevantes en la predicción en riesgo de ataques al corazón.

2. Aplicar técnicas de preprocesamiento y modelado utilizando StandardScaler, OneHotEncoder y SVC, evaluando el impacto de diferentes kernels en la frontera de decisión.

3. Optimizar el rendimiento del modelo mediante la búsqueda de hiperparámetros con GridSearchCV.


* * *

Instrucciones
-------------

### 0\. Descarga del Dataset

Utiliza el siguiente enlace para descargar el dataset de riesgos cardiovasculares desde Kaggle:

[Descargar Dataset](https://www.kaggle.com/datasets/vishardmehta/heart-risk-progression-dataset)

El dataset contiene pacientes, donde la columna `risk_category` indica el riesgo de padecer enfermedades cardiovasculares (`Low`, `Medium`, `High`)

### 1\. Exploración de datos (10%)

1.  Carga el dataset en un DataFrame de pandas.
2.  Elimina las columnas `heart_disease_risk_score` y `Patient_ID`.
3.  Realiza un pairplot entre las variables usando seaborne

### 2\. Preprocesamiento de Datos (10%)

1.  Separa las características (`X`) de la etiqueta (`Y`). La columna `risk_category` es la etiqueta.
3.  Divide el dataset en conjuntos de entrenamiento y prueba utilizando `train_test_split` de `sklearn`. **Asegúrate de usar `random_state=42` para garantizar la reproducibilidad de los resultados.**

### 3\. Exploración efecto del kernel (20%)

1. Utilice la función `plot_frontier_svc` definidia abajo para graficar la frontera de decisión entre las clases usando solo las columnas `['systolic_bp', 'age']`. Utilice **`LabelEncoder`** para codificar `Y`. Haga la gráfica para todos los kernels disponibles en sklearn.
2. Compare los resultados usando `C = [0.1, 1, 10, 1000]` y `kernel=rbf`. 
3. Responda las siguientes preguntas

- ¿Usando `rbf` que observa sucede a medida que se incrementa `C` con la frontera de decisión?

### 4\. Entrenamiento y Evaluación (30%)

1. Construye un pipeline en `sklearn` que incluya los siguientes componentes: **`StandardScaler`**, **`OneHotEncoder`** y **`SVC`**. Utilice la clase `Pipeline` de sklearn y utilice el kernel `poly` con C=`1.0`
2. Entrena el modelo con el dataset de entrenamiento
3. Producir el reporte completo de clasificación y la matriz de confusión.

### 5\. Optimizacion de parametros (30%)

1. Construye un pipeline en `sklearn` que incluya los siguientes componentes: **`StandardScaler`**, **`OneHotEncoder`** y **`SVC`**. Utilice la clase `Pipeline` de sklearn
2. Entrena el modelo con el dataset de entrenamiento usando GridSearchCV pruebe todos los kernels disponibles en sklearn y utilice `C = [0.1, 1, 10, 100]`
3. Responsa las siguientes preguntas

- ¿Cuales hiperparámetros arrojaron el mejor modelo?




In [ ]:



import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from matplotlib.colors import ListedColormap

def plot_frontier_svc(X_, y_, kernel="linear", C=1):
    """
    Grafica la frontera de decisión de un clasificador de Máquina de Vectores de Soporte (SVM)
    utilizando conjuntos de datos de entrenamiento y prueba.
    El clasificador SVM es entrenado en la versión escalada de la matriz de características.
    La función visualiza la frontera de decisión.

    :param X_: ndarray
        La matriz de características. Estos datos deben contener únicamente 2 características,
        por ejemplo, ['Glucosa', 'Edad']

    :param y_: ndarray
        Los valores objetivo/etiquetas de clase correspondientes a las filas de la matriz
        de características de entrada.

    :param kernel: str, opcional
        Especifica el tipo de kernel a utilizar en el algoritmo SVM. Por defecto es "linear".
        Otras opciones válidas incluyen "rbf", "poly", "sigmoid", etc.

    :param C: float, opcional
        Parámetro de regularización. La fuerza de la regularización es inversamente
        proporcional a C. Valor por defecto es 1.
    """

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_)

    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_scaled, y_, test_size=0.2, random_state=31)

    svm = SVC(kernel=kernel, C=C)
    svm.fit(X_train_, y_train_)

    x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
    y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    Z = svm.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.3, cmap=ListedColormap(["#ffcccc", "#cce5ff", "#d4edda"]))

    train_df = pd.DataFrame({
        'x': X_train_[:, 0],
        'y': X_train_[:, 1],
        'class': y_train_,
        'set': 'train'
    })
    test_df = pd.DataFrame({
        'x': X_test_[:, 0],
        'y': X_test_[:, 1],
        'class': y_test_,
        'set': 'test'
    })
    df = pd.concat([train_df, test_df])

    sns.scatterplot(data=df, x='x', y='y', hue='class', style='set', palette='tab10' )
    plt.xlabel("Componente Principal 1")
    plt.ylabel("Componente Principal 2")
    plt.title(f"Frontera de Decisión del SVM kernel: {kernel} C:{C}")
    plt.legend()
    plt.show()